**Bert model op basis van Bert van Universiteit van Groningen. Context handling moet nog aangepast worden. Oversampling a.d.h.v de mediaan. dynamisch treshhold zoeken voor unknown.**

In [1]:
import pandas as pd
import re
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from imblearn.over_sampling import RandomOverSampler
import torch.nn.functional as F
import os 

In [2]:
# ✅ 1. Load & Preprocess Data
script_dir = os.path.dirname(os.getcwd()) # Ga één map omhoog om 'baseline' te verwijderen en ga naar 'Data'
project_root = os.path.dirname(script_dir)  # Dit verwijdert 'baseline' van het script_dir
data_folder = os.path.join(project_root, "Data")

# 1. Dataset inladen
file_path = os.path.join(data_folder, "Grote_data_NoDupsLessThemes.xlsx")
df = pd.read_excel(file_path)

In [3]:
import pandas as pd
import re

# Drop unnecessary columns
columns_to_drop = ["file_name","question","statistical"]
df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])

# ✅ Drop rare themes (appearing < 2 times)
theme_counts = df["theme"].value_counts()
valid_themes = theme_counts[theme_counts >= 100].index
df = df[df["theme"].isin(valid_themes)]

# ✅ Drop duplicate contexts (keep first occurrence)
df = df.drop_duplicates(subset="context")

# ✅ Recompute label encoding AFTER filtering
unique_themes = list(df["theme"].unique())
theme_to_id = {theme: idx for idx, theme in enumerate(unique_themes)}
id_to_theme = {idx: theme for theme, idx in theme_to_id.items()}
df["theme_id"] = df["theme"].map(theme_to_id)

# Amount of rows
print(f"Number of rows after filtering: {len(df)}")


# Combine context and question, then clean
df["clean_text"] = ( df["clean_text"] + " " + df["context"])



Number of rows after filtering: 25398


In [4]:
print("All theme_ids:", sorted(df["theme_id"].unique()))
print("num_labels:", df["theme_id"].nunique())


All theme_ids: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
num_labels: 18


In [5]:
from sklearn.model_selection import train_test_split
import pandas as pd

# === 1. Initial full split: 70% train, 30% temp (to be split into val + test)
train_texts, temp_texts, train_labels, temp_labels = train_test_split(
    df["clean_text"].tolist(),
    df["theme_id"].tolist(),
    test_size=0.3,
    random_state=42,
    stratify=df["theme_id"]
)

# === 2. Split temp into 15% val, 15% test
val_texts, test_texts, val_labels, test_labels = train_test_split(
    temp_texts,
    temp_labels,
    test_size=0.5,  # 50% of 30% = 15%
    random_state=42,
    stratify=temp_labels
)

# === 3. Save test set to Excel
test_df = pd.DataFrame({
    "clean_text": test_texts,
    "theme_id": test_labels
})
test_df.to_excel("Test_data_HeldOut_15percent.xlsx", index=False)

# === Now you can continue using train_texts/train_labels and val_texts/val_labels as before


In [6]:
import pandas as pd

# Create a small DataFrame from train lists
train_df = pd.DataFrame({
    "clean_text": train_texts,
    "theme_id": train_labels
})

theme_counts = train_df["theme_id"].value_counts()



from collections import Counter
print("Class distribution :", Counter(train_labels))

Class distribution : Counter({9: 4438, 2: 2673, 13: 2256, 10: 2035, 4: 1507, 6: 983, 8: 725, 12: 686, 1: 605, 14: 328, 3: 288, 15: 252, 7: 224, 11: 209, 16: 207, 0: 172, 5: 130, 17: 60})


In [7]:
#remove NaN 
import pandas as pd

# Make sure to clean texts and labels together
filtered = [(x, y) for x, y in zip(train_texts, train_labels) if pd.notna(x)]
train_texts, train_labels = zip(*filtered)

# Convert back to list
train_texts = list(train_texts)
train_labels = list(train_labels)

print(f"Train set after removing NaNs: {len(train_texts)} examples")


Train set after removing NaNs: 17777 examples


In [8]:
# ✅ 7. Load BERT Tokenizer & Define Dataset Class
model_name = "GroNLP/bert-base-dutch-cased"  # pak juistee model
tokenizer = BertTokenizer.from_pretrained(model_name)

class ThemeDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx], padding="max_length", truncation=True, max_length=self.max_length, return_tensors="pt"
        )
        encoding["labels"] = torch.tensor(self.labels[idx])
        return {key: val.squeeze(0) for key, val in encoding.items()}

train_dataset = ThemeDataset(train_texts, train_labels, tokenizer)
val_dataset = ThemeDataset(val_texts, val_labels, tokenizer)

In [9]:
# ✅ 8. Load BERT Model for Classification
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=df["theme_id"].nunique())

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at GroNLP/bert-base-dutch-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
from sklearn.utils.class_weight import compute_class_weight
import torch
import numpy as np

# Compute class weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels
)

# ✅ Clip only the upper bound at 10
clipped_weights = np.clip(class_weights, None, 10.0)

# Convert to PyTorch tensor
class_weights_tensor = torch.tensor(clipped_weights, dtype=torch.float)


In [11]:
# Final check before training
print("✔️ Final weights used in training:")
for i, weight in enumerate(class_weights_tensor):
    print(f"Class {i} → {weight.item():.4f}")

✔️ Final weights used in training:
Class 0 → 5.7419
Class 1 → 1.6324
Class 2 → 0.3695
Class 3 → 3.4292
Class 4 → 0.6553
Class 5 → 7.5970
Class 6 → 1.0047
Class 7 → 4.4090
Class 8 → 1.3622
Class 9 → 0.2225
Class 10 → 0.4856
Class 11 → 4.7254
Class 12 → 1.4397
Class 13 → 0.4378
Class 14 → 3.0110
Class 15 → 3.9191
Class 16 → 4.7711
Class 17 → 10.0000


In [12]:
from transformers import Trainer
from torch.nn import CrossEntropyLoss

class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = CrossEntropyLoss(weight=class_weights_tensor.to(model.device))
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [13]:
# ✅ 9. Define Training Arguments (With Early Stopping)
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,                 # 👈 Keep only the last checkpoint
    learning_rate=2e-5,
    warmup_ratio=0.1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=8,  
    weight_decay=0.01,
    logging_dir="./logs",
    logging_strategy="epoch",  # 🔥 Log only once per epoch
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
)

# ✅ 10. Define Metrics for Evaluation
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted", zero_division=1)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# ✅ 11. Train Model with Early Stopping
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]  # Stop if no improvement for 2 epochs
)

trainer.train()

C:\Users\jefva\anaconda3\envs\bert_env\lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.832800,1.013241,0.776115,0.787714,0.776115,0.777064
2,0.847200,0.929075,0.803675,0.812404,0.803675,0.805632
3,0.461400,1.016622,0.817323,0.826531,0.817323,0.818121
4,0.235700,1.159445,0.836483,0.838265,0.836483,0.836794
5,0.118300,1.298289,0.840420,0.841478,0.840420,0.840354
6,0.052700,1.406985,0.845669,0.846685,0.845669,0.845455
7,0.023700,1.468572,0.840157,0.840249,0.840157,0.839207
8,0.013000,1.470433,0.845144,0.845295,0.845144,0.844681


TrainOutput(global_step=17784, training_loss=0.4481062143705903, metrics={'train_runtime': 3920.8026, 'train_samples_per_second': 36.272, 'train_steps_per_second': 4.536, 'total_flos': 3.742397731351757e+16, 'train_loss': 0.4481062143705903, 'epoch': 8.0})

In [14]:
trainer.save_model("output/grobertd30_context")#naam


In [16]:
import os
from datetime import datetime
import numpy as np
import pandas as pd
import json


# === Create timestamped save path in OneDrive ===
run_id = datetime.now().strftime("%Y-%m-%d_%H-%M")
save_path = rf"C:\Users\jefva\Documents\Master\Thesis_s2\results\mBERT\Run_{run_id}"
os.makedirs(save_path, exist_ok=True)

# === Save model using Trainer ===
trainer.save_model(save_path)

# === Get predictions ===
preds_output = trainer.predict(val_dataset)
logits = preds_output.predictions
predictions = np.argmax(logits, axis=1)

# === Save predictions to CSV ===
output_df = pd.DataFrame({
    "text": val_texts,  # make sure val_texts is defined
    "true_label": val_labels,  # make sure val_labels is defined
    "predicted_label": predictions,
    "logits": logits.tolist()
})

csv_path = os.path.join(save_path, "val_predictions.csv")
output_df.to_csv(csv_path, index=False)

# === Optional: Save a description file ===
description = """
Model: GroNLP BERT-based Dutch Cased
Training Details:
geen context meegegeven in zowel train als val
5 epochs 

data set : Grote_data_NoDupsLessThemesENnoWords9
"""

desc_path = os.path.join(save_path, "model_description.txt")
with open(desc_path, "w", encoding="utf-8") as f:
    f.write(description)

# === Save label mappings ===
mappings_path = os.path.join(save_path, "label_mappings.json")
with open(mappings_path, "w", encoding="utf-8") as f:
    json.dump({
        "theme_to_id": theme_to_id,
        "id_to_theme": {str(k): v for k, v in id_to_theme.items()},  # keys must be str for JSON
        "unique_themes": unique_themes  # Save the unique themes list

    }, f, ensure_ascii=False, indent=4)


print(f"Everything saved in: {save_path}")


Everything saved in: C:\Users\jefva\Documents\Master\Thesis_s2\results\mBERT\Run_2025-04-29_12-19
